# Final Project: Can AI be Ethical?
Khushi Magiawala, Rahul Rajan, Shruthi Sundar

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
try:
 from google.colab import drive
  IN_COLAB=True
except:
 IN_COLAB=False

if IN_COLAB:
 print("We're running Colab")

IndentationError: unexpected indent (<ipython-input-4-2f2224ff3fb6>, line 3)

In [6]:
%pip install jsonlines


The following command must be run outside of the IPython shell:

    $ pip install jsonlines

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [1]:
import jsonlines
import pandas as pd

def prepare_data(filename):
  df = pd.DataFrame()
  def rename_field(obj):
    key = 'immoral_action' if 'immoral_action' in obj else 'moral_action'
    obj['action'] = obj.pop(key)
    # print(obj)
    return obj
  with jsonlines.open(filename) as f:
      data = [rename_field(x) for x in f.iter()]
      df = pd.DataFrame(data)
  print(df['label'])
  return df
    

In [22]:
train_df = prepare_data('train.jsonl')
test_df = prepare_data('test.jsonl')
dev_df = prepare_data('dev.jsonl')

train_df['label'].apply(lambda x: 0 if x == '0' else 1)
test_df['label'].apply(lambda x: 0 if x == '0' else 1)
dev_df['label'].apply(lambda x: 0 if x == '0' else 1)
print(train_df)

0        1
1        0
2        1
3        0
4        1
        ..
19995    0
19996    1
19997    0
19998    1
19999    0
Name: label, Length: 20000, dtype: object
0       1
1       0
2       1
3       0
4       1
       ..
1995    0
1996    1
1997    0
1998    1
1999    0
Name: label, Length: 2000, dtype: object
0       1
1       0
2       1
3       0
4       1
       ..
1995    0
1996    1
1997    0
1998    1
1999    0
Name: label, Length: 2000, dtype: object
                                    ID  \
0      37XITHEISX7HXBGAFQTNMO7R7TSRCQ1   
1      37XITHEISX7HXBGAFQTNMO7R7TSRCQ0   
2      3TDXMTX3CCSFOV4QFCMBMWEQPJGI6K1   
3      3TDXMTX3CCSFOV4QFCMBMWEQPJGI6K0   
4      3QAPZX2QN5BGZZ49OKL2QFXIMJ502V1   
...                                ...   
19995  3LO69W1SU4BJBP12CSSAQ3J70IWGLF0   
19996  3HOSI13XH0MX1T3JXLTE1585EO1DDN1   
19997  3HOSI13XH0MX1T3JXLTE1585EO1DDN0   
19998  3CFJTT4SXUOYSVIVYMR3SAEMHEOI781   
19999  3CFJTT4SXUOYSVIVYMR3SAEMHEOI780   

                              

In [5]:
print(train_df.columns)

Index(['ID', 'norm', 'situation', 'intention', 'moral_consequence', 'label',
       'action', 'immoral_consequence'],
      dtype='object')


In [6]:
import torch
import random
import numpy as np
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
import pandas as pd
from src.preprocess import clean_text 
import nltk
from tqdm import tqdm

nltk.download('punkt')
train_df["action_tokenized"] = train_df["action"].apply(lambda x: nltk.word_tokenize(clean_text(x.lower())))
test_df["action_tokenized"] = test_df["action"].apply(lambda x: nltk.word_tokenize(clean_text(x.lower())))
dev_df["action_tokenized"] = dev_df["action"].apply(lambda x: nltk.word_tokenize(clean_text(x.lower())))

train_df["situation_tokenized"] = train_df["situation"].apply(lambda x: nltk.word_tokenize(clean_text(x.lower())))
test_df["situation_tokenized"] = test_df["situation"].apply(lambda x: nltk.word_tokenize(clean_text(x.lower())))
dev_df["situation_tokenized"] = dev_df["situation"].apply(lambda x: nltk.word_tokenize(clean_text(x.lower())))

train_df["norm_tokenized"] = train_df["norm"].apply(lambda x: nltk.word_tokenize(clean_text(x.lower())))
test_df["norm_tokenized"] = test_df["norm"].apply(lambda x: nltk.word_tokenize(clean_text(x.lower())))
dev_df["norm_tokenized"] = dev_df["norm"].apply(lambda x: nltk.word_tokenize(clean_text(x.lower())))

[nltk_data] Downloading package punkt to /Users/rajan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
from src.dataset import split_train_val_test, generate_vocab_map


In [11]:
train_vocab, reverse_vocab = generate_vocab_map(train_df)
test_vocab, reverse_vocab = generate_vocab_map(test_df)
dev_vocab, reverse_vocab = generate_vocab_map(dev_df)

action_vocab, __ = generate_vocab_map(train_df)
situation_vocab, ___ = generate_vocab_map(train_df)
intention_vocab, __ = generate_vocab_map(train_df)
train_df['label'] = train_df['label'].apply(lambda x:np.int64(int(x)))
test_df['label'] = test_df['label'].apply(lambda x:np.int64(int(x)))
dev_df['label'] = dev_df['label'].apply(lambda x:np.int64(int(x)))

In [13]:
from src.dataset import MoralStoriesDataset
from torch.utils.data import RandomSampler

train_dataset = MoralStoriesDataset(train_vocab, train_df)
val_dataset = MoralStoriesDataset(train_vocab, dev_df)
test_dataset = MoralStoriesDataset(train_vocab, test_df)

train_sampler = RandomSampler(train_dataset)
val_sampler = RandomSampler(val_dataset)
test_sampler = RandomSampler(test_dataset)

In [14]:
from torch.utils.data import DataLoader
from src.dataset import collate_fn
BATCH_SIZE = 64

train_iterator = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=train_sampler, collate_fn=collate_fn)
val_iterator = DataLoader(val_dataset, batch_size=BATCH_SIZE, sampler=val_sampler, collate_fn=collate_fn)
test_iterator = DataLoader(test_dataset, batch_size=BATCH_SIZE, sampler=test_sampler, collate_fn=collate_fn)

Modeling

In [15]:
for a,s,n,l in test_iterator:
    print(a,s,n,l)
test_iterator = DataLoader(test_dataset, batch_size=BATCH_SIZE, sampler=test_sampler, collate_fn=collate_fn)

tensor([[1174,   51,    8,  ...,    0,    0,    0],
        [ 443, 1058,   14,  ...,    0,    0,    0],
        [ 812,  171,   99,  ...,    0,    0,    0],
        ...,
        [ 307,  165,    4,  ...,    0,    0,    0],
        [   1,  574,  229,  ...,    0,    0,    0],
        [5356,  165,    4,  ...,    0,    0,    0]]) tensor([[1174,   71,  213,  ...,    0,    0,    0],
        [ 443,   86, 5295,  ...,    0,    0,    0],
        [ 812,  655,   49,  ...,    0,    0,    0],
        ...,
        [ 307,   10,   49,  ...,    0,    0,    0],
        [  73, 3316, 3655,  ...,    0,    0,    0],
        [5356,  126, 4950,  ...,    0,    0,    0]]) tensor([[ 119,   71,  333,    4,   92, 5037,  571,    4,   47, 1305,  943,   16],
        [ 215,  260,   20,  149,  293, 5684,  400,  215,  252, 4554,  119,   16],
        [ 119,   71,  279,    4, 3571,  314,  600,  372, 1104,    1,   16,    0],
        [ 119,   71, 4376,    4,  455,   84, 1510,  877, 6583,   16,    0,    0],
        [1025,   14,

In [19]:
import numpy as np
import gensim.downloader as api

def download_word2vec_embeddings():
    print("Downloading pre-trained word embeddings from: word2vec-google-news-300.\n" 
          + "Note: This can take a few minutes.\n")
    wv = api.load("word2vec-google-news-300")
    print("\nLoading complete!\n" +
          "Vocabulary size: {}".format(len(wv.vocab)))
    return wv

word_vectors = download_word2vec_embeddings()

Note: This can take a few minutes.



FileNotFoundError: [Errno 2] No such file or directory: '/Users/rajan/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz'

In [ ]:
EMBEDDING_VECTOR_LENGTH = 300
import itertools

def construct_embedding_matrix(vocab):
  embedding_dict = {}
  c = 0
  
  for word in vocab.keys():
    if word in word_vectors:
      vector = word_vectors[word]
      embedding_dict[word] = vector
    else:
      c+=1


  num_words = len(vocab) + 1
  embedding_matrix=np.zeros((num_words, EMBEDDING_VECTOR_LENGTH))
  for word,i in tqdm(vocab.items()):
    vect=embedding_dict.get(word, [])
    if len(vect)>0:
        embedding_matrix[i] = vect[:EMBEDDING_VECTOR_LENGTH]
  return embedding_matrix

action_embedding_matrix = construct_embedding_matrix(action_vocab)
situation_embedding_matrix = construct_embedding_matrix(situation_vocab)
intention_embedding_matrix = construct_embedding_matrix(intention_vocab)

(8312, 300)


100%|██████████| 8311/8311 [00:00<00:00, 460201.20it/s]


(8312, 300)


100%|██████████| 8311/8311 [00:00<00:00, 420665.42it/s]


(8312, 300)


100%|██████████| 8311/8311 [00:00<00:00, 431517.67it/s]


In [ ]:
from src.models import ClassificationModel

 
model = ClassificationModel(len(action_vocab),len(situation_vocab), len(intention_vocab), action_embedding_matrix, situation_embedding_matrix, intention_embedding_matrix, 
                            embedding_dim=300, hidden_dim=16, num_layers=2, bidirectional=True)
model = model.to(device)

In [ ]:
from torch.optim import AdamW

criterion, optimizer = None, None
criterion = torch.nn.BCELoss()
optimizer = AdamW([param for param in model.parameters() if param.requires_grad == True], lr=0.003)
# train

In [ ]:

# returns the total loss calculated from criterion
def train_loop(model, criterion, iterator):
    model.train()
    total_loss = 0
    true, pred = [], []
    for a, s, n, y in tqdm(iterator):
        optimizer.zero_grad()
        outputs = model(a.squeeze().to(device), s.squeeze().to(device), n.squeeze().to(device))
        loss = criterion(outputs.squeeze(), y.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        true.extend(y.cpu() > 0.5)
        pred.extend(outputs.cpu() > 0.5)
    print(f"TRAIN F-1: {binary_macro_f1(true, pred)}")
    print(f"TRAIN ACC: {accuracy(true, pred)}")
    return total_loss
    
def val_loop(model, criterion, iterator):
    true, pred = [], []
    model.eval()
    with torch.no_grad():
        for a, s, n, y in tqdm(iterator):
            output = model(a.squeeze().to(device), s.squeeze().to(device), n.squeeze().to(device))
            # convert output to boolean
            output = output.cpu() > 0.5
            # convert y to boolean
            y = y > 0.5
            true.extend(y)
            pred.extend(output)        
    return true, pred

In [ ]:
from src.eval_utils import binary_macro_f1, accuracy
print(action_embedding_matrix.shape[0])
print(len(action_vocab))
true, pred = val_loop(model, criterion, val_iterator)
print(binary_macro_f1(true, pred))
print(accuracy(true, pred))

8312
8311


100%|██████████| 32/32 [00:01<00:00, 18.21it/s]


0.19980015988008795
0.5


In [ ]:
TOTAL_EPOCHS = 32

for epoch in range(TOTAL_EPOCHS):
    train_loss = train_loop(model, criterion, train_iterator)
    true, pred = val_loop(model, criterion, val_iterator)
    print(f"EPOCH: {epoch}")
    print(f"TRAIN LOSS: {train_loss}")
    print(f"VAL F-1: {binary_macro_f1(true, pred)}")
    print(f"VAL ACC: {accuracy(true, pred)}")

100%|██████████| 313/313 [00:20<00:00, 15.18it/s]


TRAIN F-1: 0.38213270562201523
TRAIN ACC: 0.6682


100%|██████████| 32/32 [00:01<00:00, 18.45it/s]


EPOCH: 0
TRAIN LOSS: 188.27190735936165
VAL F-1: 0.3891053553544145
VAL ACC: 0.678


100%|██████████| 313/313 [00:20<00:00, 15.37it/s]


...

EPOCH: 30
TRAIN LOSS: 20.744258527178317
VAL F-1: 0.39209766326043755
VAL ACC: 0.681


100%|██████████| 313/313 [00:18<00:00, 16.72it/s]


TRAIN F-1: 0.6480792841757422
TRAIN ACC: 0.97915


100%|██████████| 32/32 [00:01<00:00, 20.41it/s]

EPOCH: 31
TRAIN LOSS: 17.70276849216316
VAL F-1: 0.39575173417060006
VAL ACC: 0.6855


Test Metrics

In [ ]:
true, pred = val_loop(model, criterion, test_iterator)
print(f"TEST F-1: {binary_macro_f1(true, pred)}")
print(f"TEST ACC: {accuracy(true, pred)}")

100%|██████████| 32/32 [00:01<00:00, 20.29it/s]


TEST F-1: 0.3684843859745132
TEST ACC: 0.652


#### 2. Baseline Models 

The below section details implementation of our baseline models- a Naive Bayes classifier and Logistic Regression classifier

In [24]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"
import matplotlib.pyplot as plt
%matplotlib inline
import re
import string
import jsonlines
pd.set_option('display.max_colwidth', -1)
import seaborn as sns
import nltk
import pandas as pd
nltk.download('wordnet')
#import scattertext as st
#import spacy

from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

<ipython-input-24-94d22c140184>:8: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)
[nltk_data] Downloading package wordnet to /Users/rajan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

/usr/local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [25]:
def filler(text): 
    if text.dtype is pd.np.dtype(float): #if in the column the data type is a float/int, it replaces w an empty string
        return text.fillna('')
    elif text.dtype is pd.np.dtype(int): 
        return text.fillna('')
    else:
        return text.fillna('NA')

In [26]:
train_df = train_df.apply(filler)
test_df = test_df.apply(filler)
dev_df = dev_df.apply(filler)

<ipython-input-25-8e7d9e16f3db>:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  if text.dtype is pd.np.dtype(float): #if in the column the data type is a float/int, it replaces w an empty string
<ipython-input-25-8e7d9e16f3db>:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  elif text.dtype is pd.np.dtype(int):


In [ ]:
train_df.action=train_df.action.str.replace('[^a-zA-Z]', ' ')
test_df.action=train_df.action.str.replace('[^a-zA-Z]', ' ')
dev_df.action=train_df.action.str.replace('[^a-zA-Z]', ' ')

In [27]:
RE_PREPROCESS = r'\W+|\d+'  #raw string + alphanumeric character + one or more digits ( regular expressions)
train_df.action = np.array( [ re.sub(RE_PREPROCESS, ' ', answer).lower() for answer in train_df.action])
test_df.action = np.array( [ re.sub(RE_PREPROCESS, ' ', answer).lower() for answer in test_df.action])
dev_df.action = np.array( [ re.sub(RE_PREPROCESS, ' ', answer).lower() for answer in dev_df.action])

In [28]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
#WordNetLemmatizer returns the input word unchanged if it cannot be found in WordNet.
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics

In [30]:
from sklearn.model_selection  import train_test_split
#train_df['label'] = train_df['label'].apply(lambda x: 0 if x == '0' else 1)
X = train_df.action
y = train_df['label']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,stratify=train_df['label'],random_state=2,shuffle=True)

train_df.head(5)

,ID,norm,situation,intention,moral_consequence,label,action,immoral_consequence
0,37XITHEISX7HXBGAFQTNMO7R7TSRCQ1,It's good to do activities together with your spouse.,Jenny's husband asks her to go hiking with him like they used to do.,Jenny wants to tell her husband if she'll go hiking with him.,Jenny's husband is happy that they still share activities together and thinks the hike will be a chance to reconnect.,1,jenny agrees to go hiking with her husband and seems excited for the activity,NA
1,37XITHEISX7HXBGAFQTNMO7R7TSRCQ0,It's good to do activities together with your spouse.,Jenny's husband asks her to go hiking with him like they used to do.,Jenny wants to tell her husband if she'll go hiking with him.,NA,0,jenny says she doesn t feel like going hiking with her husband and will just stay home,Jenny's husband feels like they don't do things together any more and that they are growing apart.
2,3TDXMTX3CCSFOV4QFCMBMWEQPJGI6K1,It's good to show remorse for your actions,Claire is walking through the mall when she realizes that she has unintentionally let the door slam in a lady's face.,Claire wants to quickly get to her intended store.,The lady accepts Claire's apology.,1,claire reaches back opens the door and apologizes to the lady before heading on her way,NA
3,3TDXMTX3CCSFOV4QFCMBMWEQPJGI6K0,It's good to show remorse for your actions,Claire is walking through the mall when she realizes that she has unintentionally let the door slam in a lady's face.,Claire wants to quickly get to her intended store.,NA,0,claire looks away from the lady and continues to her destination,The lady thinks that Claire is a rude person.
4,3QAPZX2QN5BGZZ49OKL2QFXIMJ502V1,It's rude to lock someone in the bathroom.,Timothy lives alone with his dog in his house who is barking loudly because he wants to go out and walk.,Timothy doesn't want to hear his dog bark anymore.,Timothy's dog stops barking and is happier and healthy.,1,timothy puts a leash on his dog and takes him for a walk around the neighborhood,NA


In [31]:
get_ipython().system('pip install nltk')
nltk.download('punkt')
get_ipython().system('pip install scikit-learn')

vect=TfidfVectorizer(tokenizer=LemmaTokenizer(),stop_words='english',ngram_range=(1, 2))
vect.fit(X_train)

train=vect.transform(X_train) # transforming from string to vector (train and test models)
test=vect.transform(X_test)

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.


[nltk_data] Downloading package punkt to /Users/rajan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.


/usr/local/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
/usr/local/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


TfidfVectorizer(ngram_range=(1, 2), stop_words='english',
                tokenizer=<__main__.LemmaTokenizer object at 0x150ecba30>)

In [33]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn import metrics

logreg = LogisticRegression(penalty='l2', C=0.2)
logreg.fit(train, y_train)
y_pred = logreg.predict(test)

LogisticRegression(C=0.2)

In [34]:
y_true = y_test
#y_pred = output_log
precision_recall_fscore_support(y_true, y_pred, average='macro')
accuracy_score(y_true, y_pred)
#train_df[train_df['label']=='1']

(0.6927896664738771, 0.692, 0.6916842847075406, None)

0.692

In [35]:
nb = MultinomialNB(alpha=0.2)
nb.fit(train, y_train)
output_nb = nb.predict(test)

MultinomialNB(alpha=0.2)

In [40]:
y_true = y_test
y_pred = output_nb
precision_recall_fscore_support(y_true, y_pred, average='macro')
accuracy_score(y_true, y_pred)

(0.6565001565001565, 0.6565000000000001, 0.6564999141249785, None)

0.6565

In [37]:
rf = RandomForestClassifier(n_estimators = 25)
rf.fit(train,y_train)
output_rf = rf.predict(test)

RandomForestClassifier(n_estimators=25)

In [38]:
y_true = y_test
y_pred = output_rf
precision_recall_fscore_support(y_true, y_pred, average='macro')
accuracy_score(y_true, y_pred)

(0.6650237634219327, 0.665, 0.6649879395658245, None)

0.665